In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import  DecisionTreeClassifier,export_graphviz
from xgboost import plot_importance
from matplotlib import pyplot as plt
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

# 一、 预处理

## 读取训练集以及训练集的数据并替换数据，数值0替换为空值

In [3]:
data = pd.read_csv("train.csv")
data1 = data.replace(0,np.nan)
data1.info(verbose=True,null_counts=True)

C:\Users\yaokexin\AppData\Local\Temp\ipykernel_18000\2675975050.py:1: DtypeWarning: Columns (48,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("train.csv")
C:\Users\yaokexin\AppData\Local\Temp\ipykernel_18000\2675975050.py:3: FutureWarning: null_counts is deprecated. Use show_counts instead
  data1.info(verbose=True,null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23432 entries, 0 to 23431
Data columns (total 657 columns):
 #    Column                 Non-Null Count  Dtype  
---   ------                 --------------  -----  
 0    pax_name               23432 non-null  object 
 1    pax_passport           23432 non-null  object 
 2    seg_route_from         23432 non-null  object 
 3    seg_route_to           23432 non-null  object 
 4    seg_flight             23432 non-null  object 
 5    seg_cabin              23431 non-null  object 
 6    seg_dep_time           23432 non-null  object 
 7    pax_fcny               23398 non-null  float64
 8    pax_tax                23342 non-null  float64
 9    emd_lable              7202 non-null   float64
 10   emd_lable2             1475 non-null   float64
 11   gender                 23432 non-null  object 
 12   age                    23432 non-null  object 
 13   birth_date             23432 non-null  object 
 14   residence_country      23432 non-nul

In [4]:
test = pd.read_csv("test.csv")
test1 = test.replace(0,np.nan)
test1.info(verbose=True,null_counts=True)

C:\Users\yaokexin\AppData\Local\Temp\ipykernel_18000\3008618231.py:1: DtypeWarning: Columns (46,47,56,57,141,142,266,340,360,437,650,651) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv("test.csv")
C:\Users\yaokexin\AppData\Local\Temp\ipykernel_18000\3008618231.py:3: FutureWarning: null_counts is deprecated. Use show_counts instead
  test1.info(verbose=True,null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6771 entries, 0 to 6770
Data columns (total 657 columns):
 #    Column                 Non-Null Count  Dtype  
---   ------                 --------------  -----  
 0    pax_name               6771 non-null   object 
 1    pax_passport           6771 non-null   object 
 2    seg_route_from         6771 non-null   object 
 3    seg_route_to           6771 non-null   object 
 4    seg_flight             6771 non-null   object 
 5    seg_cabin              6771 non-null   object 
 6    seg_dep_time           6771 non-null   object 
 7    pax_fcny               6767 non-null   float64
 8    pax_tax                6770 non-null   float64
 9    emd_lable              0 non-null      float64
 10   emd_lable2             0 non-null      float64
 11   gender                 6771 non-null   object 
 12   age                    6771 non-null   object 
 13   birth_date             6771 non-null   object 
 14   residence_country      6771 non-null  

## 将训练集以及验证集的原本的空值替换为数值0，保留除了object和datetime类型的之外的特征因子

In [5]:
# 原本的空值替换为数值0，保留除了object和datetime类型的之外的特征因子
data1['seg_route_to'] = data1['seg_route_to'].map({'JFK': 1, 'LAX':2, 'SYD':3})
data1['seg_cabin'] = data1['seg_cabin'].map({'F': 1, 'J':2, 'W':3,'Y':4,'C':5})
data1['seg_flight'] = data1['seg_flight'].map({'AB1000':0,'AB1001': 1,'AB1002':2,'AB1003':3, 'AB1004':4, 'AB1005':5,'AB1006':6,'AB1007': 7, 'AB1008':8,'AB1009':9, 'AB1010':10,'AB1011':11,'AB1012':12,'AB1013':13, 'AB1014':14,'AB1015':15, 'AB1016':16})

data2 = data1.replace(np.nan,0)
data2 = data2.select_dtypes(exclude=['object','datetime64[ns]'])
data2 = data2.drop(['emd_lable'],axis=1)
data2['pax_name']=data1['pax_name']
data2.info(verbose=True,null_counts=True)
data2.to_csv('D:/XGboost/trainh.csv',encoding='utf-8',index=False)

C:\Users\yaokexin\AppData\Local\Temp\ipykernel_18000\3983861325.py:10: FutureWarning: null_counts is deprecated. Use show_counts instead
  data2.info(verbose=True,null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23432 entries, 0 to 23431
Data columns (total 527 columns):
 #    Column                 Non-Null Count  Dtype  
---   ------                 --------------  -----  
 0    seg_route_to           23432 non-null  int64  
 1    seg_flight             23432 non-null  int64  
 2    seg_cabin              23432 non-null  float64
 3    pax_fcny               23432 non-null  float64
 4    pax_tax                23432 non-null  float64
 5    emd_lable2             23432 non-null  float64
 6    ffp_nbr                23432 non-null  float64
 7    often_city             23432 non-null  float64
 8    cabin_hf_cnt_m3        23432 non-null  float64
 9    cabin_hf_cnt_m6        23432 non-null  float64
 10   cabin_hf_cnt_y1        23432 non-null  float64
 11   cabin_hf_cnt_y2        23432 non-null  float64
 12   cabin_hf_cnt_y3        23432 non-null  float64
 13   cabin_f_cnt_m3         23432 non-null  float64
 14   cabin_f_cnt_m6         23432 non-nul

In [6]:
# 验证集原本的空值替换为数值0，保留除了object和datetime类型的之外的特征因子
test1['seg_route_to'] = test1['seg_route_to'].map({'JFK': 1, 'LAX':2, 'SYD':3})
test1['seg_cabin'] = test1['seg_cabin'].map({'F': 1, 'J':2, 'W':3,'Y':4,'C':5})
test1['seg_flight'] = test1['seg_flight'].map({'AB1000':0,'AB1001': 1,'AB1002':2,'AB1003':3, 'AB1004':4, 'AB1005':5,'AB1006':6,'AB1007': 7, 'AB1008':8,'AB1009':9, 'AB1010':10,'AB1011':11,'AB1012':12,'AB1013':13, 'AB1014':14,'AB1015':15, 'AB1016':16})

test2 = test1.replace(np.nan,0)
test2 = test2.select_dtypes(exclude=['object','datetime64[ns]'])
test2 = test2.drop(['emd_lable'],axis=1)
test2['pax_name']=test1['pax_name']
test2.info(verbose=True,null_counts=True)
data2.to_csv('D:/XGboost/testh.csv',encoding='utf-8',index=False)

C:\Users\yaokexin\AppData\Local\Temp\ipykernel_18000\851905906.py:10: FutureWarning: null_counts is deprecated. Use show_counts instead
  test2.info(verbose=True,null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6771 entries, 0 to 6770
Data columns (total 511 columns):
 #    Column                 Non-Null Count  Dtype  
---   ------                 --------------  -----  
 0    seg_route_to           6771 non-null   int64  
 1    seg_flight             6771 non-null   int64  
 2    seg_cabin              6771 non-null   int64  
 3    pax_fcny               6771 non-null   float64
 4    pax_tax                6771 non-null   float64
 5    emd_lable2             6771 non-null   float64
 6    cabin_hf_cnt_m3        6771 non-null   float64
 7    cabin_hf_cnt_m6        6771 non-null   float64
 8    cabin_hf_cnt_y1        6771 non-null   float64
 9    cabin_hf_cnt_y2        6771 non-null   float64
 10   cabin_hf_cnt_y3        6771 non-null   float64
 11   cabin_f_cnt_m3         6771 non-null   float64
 12   cabin_f_cnt_m6         6771 non-null   float64
 13   cabin_f_cnt_y1         6771 non-null   float64
 14   cabin_f_cnt_y2         6771 non-null  